In [ ]:
import cv2
import os
import numpy as np
from gtts import gTTS

In [ ]:
objects = []
directions = []

In [ ]:
def illustrate(img, class_id, confidence, x, y, x_plus_w, y_plus_h):
    label = str(classes[class_id])
    color = COLORS[class_id]
    cv2.rectangle(img, (x,y), (x_plus_w,y_plus_h), color, 2)
    cv2.putText(img, label, (x-10,y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    objects.append(label)

In [ ]:
def get_output_layers(net):
    layer_names = net.getLayerNames()
    output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
    return output_layers

In [ ]:
image = cv2.imread('images/example4.jpeg')
Width = image.shape[1]
Height = image.shape[0]
scale = 0.00392
classes = None

In [ ]:
with open('yolov3.txt', 'r') as f:
    classes = [line.strip() for line in f.readlines()]

In [ ]:
COLORS = np.random.uniform(0, 255, size=(len(classes), 3))
net = cv2.dnn.readNet('yolov3.weights', 'yolov3.cfg')
blob = cv2.dnn.blobFromImage(image, scale, (416,416), (0,0,0), True, crop=False)
net.setInput(blob)
outs = net.forward(get_output_layers(net))

In [ ]:
class_ids = []
confidences = []
boxes = []
conf_thresh = 0.5
nms_thresh = 0.4


In [ ]:
for out in outs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > 0.5:
            center_x = int(detection[0] * Width)
            center_y = int(detection[1] * Height)
            w = int(detection[2] * Width)
            h = int(detection[3] * Height)
            x = center_x - w / 2
            y = center_y - h / 2
            class_ids.append(class_id)
            confidences.append(float(confidence))
            boxes.append([x, y, w, h])

In [ ]:
indices = cv2.dnn.NMSBoxes(boxes, confidences, conf_thresh, nms_thresh)
for i in indices:
    i = i
    box = boxes[i]
    x = box[0]
    y = box[1]
    w = box[2]
    h = box[3]
    print(str(classes[class_ids[i]]) + ": " + str(confidences[i]))
    illustrate(image, class_ids[i], confidences[i], round(x), round(y), round(x+w), round(y+h))
    # print(str(classes[class_id[i]]))

In [ ]:
cv2.imshow("object detection", image)
cv2.waitKey()
cv2.imwrite("object-detection.jpg", image)
cv2.destroyAllWindows()

In [ ]:
obj_str = ""
for i in objects:
    obj_str += i
    obj_str += " "


In [ ]:
tts = gTTS('Objects are, ' + obj_str)
tts.save('environment.mp3')

In [ ]:
os.system("environment.mp3")